In [1]:
from http import client
from urllib import response
from slack_sdk import WebClient
import pandas as pd
import json

pd.options.display.max_colwidth = 100

client = WebClient(token ="") # ¡¡You need to insert a Slack app token here for this script to work!!

objective = 100000000000 # Max number of messages that will possibly be checked
query_size = 200 # Max number of messages per query
queries = int(objective/query_size) # Number of queries that will be attempted at most.
cursor = None # Marker for the data the next query will ask
for i in range(queries):
    
    response = client.conversations_history( # This method queries a given channel and stores the response in a variable. It usually needs to be executed several times 
        channel="C033KQ2V37V", # because the maximum query size is 1000.
        limit=query_size,
        cursor=cursor,
        oldest=1654030800

    )
    conversation_history = response["messages"] # We store the message data into a new variable 
    
    
    if i == 0: # The following lines create a list or append new results to the growing list of messages
        full_conversation = conversation_history
    else:
        full_conversation = full_conversation + conversation_history

    print(i*query_size) # Prints the progress of the querying
    if response["response_metadata"] != None: # The result of the query tells us exactly the next cursor we should use to get the next batch of messages
        cursor = response["response_metadata"]["next_cursor"]
    else: # Once there are no more messages to query, the loop stops
        break  


with open("reviews_jun_2022-2023.json", "w") as f: # We open a new file in json format (it is basically a dictionary or a list)
        json.dump(full_conversation, f) # We load our full results there

0
200
400
600
800
1000


In [2]:
import json
import pandas as pd
import re
import datetime

with open("reviews_jun_2022-2023.json", "r") as f: # We open the file we created in extract.py
    e = json.load(f) # We load its content into a variable. The content is a list of dictionaries. They are the messages and their data

l = []
for i in e: # We add tuples to a list specifying if they belong to airtable or appbot. We do that by checking if the word appbot or airtable appear in each mesaage.
    if isinstance(i["text"], dict): # If you study the structure of the data in the file, you can deduce how this loop works.
        l+=[i]
    else:
        if "airtable" in i["text"]:
            l+=[(i["ts"],json.dumps(i), i["text"], "airtable")]
        elif "appbot" in i["text"]:
            l+=[(i["ts"],json.dumps(i), i["text"], "appbot")]
            #print(i)
        else:
            if "blocks" in i.keys():
                if "appbot" in "".join([o["text"]["text"] for o in i["blocks"] if "text" in o.keys() ]):
                    l+=[(i["ts"],json.dumps(i), "".join([o["text"]["text"] if "text" in o.keys() else o["elements"][0]["text"] if "elements" in o.keys() and "text" in o["elements"][0].keys() else "" for o in i["blocks"]  ]), "appbot")]
                    

data = pd.DataFrame(l, columns = ["ts", "original_record","text", "bot"]) # We transform the labeled data into a new pandas Dataframe

# The following lines extract every single field we need from the message raw text, using references like emojis and words.
# It uses an additional special universal language known as Regex, which is essential for parsing text.

data["strings"] = data["text"].str.extract(":speaking_head_in_silhouette:(.*)")
data["strings"] = data["strings"].fillna("")

data["appbot"] = data["text"].str.extract(r"[★☆]{5}\n(.*)\n", re.DOTALL)
data.loc[data["appbot"].str.contains("English Translation").fillna(False),"appbot"] = ""
data["appbot"] = data["appbot"].fillna("")

data["appbot_eng"] = data["text"].str.extract(r"English Translation_\n(.*)-{19}\n", re.DOTALL) 
data.loc[~data["appbot_eng"].str.contains("").fillna(False),"appbot_eng"] = ""

data["Opinion"] = data["appbot"] + data["strings"]
data["Opinion"] = data["Opinion"] + data["appbot_eng"]

data["sentiment_airtable"] = data["text"].str.extract("Sentiment:  (.*)").fillna("")
data["sentiment_appbot"] = data["text"].str.extract(":.+circle: ([A-Z][a-z]+)").fillna("")

data["sentiment"] = data["sentiment_appbot"] + data["sentiment_airtable"]

data = data.drop(["strings", "appbot", "appbot_eng", "sentiment_airtable", "sentiment_appbot"], axis = 1)

data["topic"] = data["text"].str.extract("Topic :jigsaw:  (.*)").fillna("")

# We convert the timestamp of the message from a number to a readable date and time.
data["ts"] = [datetime.datetime.fromtimestamp(float(i)).strftime("%Y-%m-%d %H:%M:%S") for i in data["ts"]]

data.to_csv("reviews_jun_2022-2023_raw.csv") # We export the raw results to an excel

data = data[data["Opinion"] != ""] # We remove the completely empty opinion cells because Appbot messages usually consist of a general info message and 
# an opinion message. We only want the opinion messages.

data = data.drop(["original_record", "text"], axis = 1) # We remove the raw columns from the data


data.to_csv("reviews_jun_2022-2023_clean.csv") # We export a cleaner alternative dataset